## [실습 환경 Setting]
### 1. RealESRGAN 설치
- Download RealESRGAN pjt :
    ```
    pip install basicsr
    pip install facexlib
    pip install gfpgan
    git clone https://github.com/xinntao/Real-ESRGAN.git
    <python_home>/Lib/site-packages 위치에 Real-ESRGAN directory 통 copy
    cd <python_home>/Lib/site-packages/Real-ESRGAN
    pip install -r requirements.txt
    python setup.py develop
    ```
### 2. RealESRGAN 용 AI Model 설치
- Download RealESRGAN model(Restoring model):
    https://github.com/sczhou/CodeFormer/releases/download/v0.1.0/codeformer.pth
    
    저장 위치 : python code 에 model 위치 명시

In [ ]:
import torch
import cv2
import numpy as np
from basicsr.archs.rrdbnet_arch import RRDBNet
from realesrgan import RealESRGANer

def upscale_image(input_path, output_path, model_path, scale=2):
    # 장치 설정
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    # 모델 생성
    model = RRDBNet(num_in_ch=3, num_out_ch=3, num_feat=64, num_block=23, num_grow_ch=32, scale=scale)
    
    # 업스케일러 생성 (half=False로 설정하여 FP32 사용)
    upscaler = RealESRGANer(scale=scale, model_path=model_path, model=model, tile=400, tile_pad=10, pre_pad=0, half=False)
 
    # 이미지 로드 (OpenCV 사용)
    img = cv2.imread(input_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # BGR에서 RGB로 변환
    
    # 업스케일링 수행
    output, _ = upscaler.enhance(img, outscale=scale)
    
    # 결과 저장 (OpenCV 사용)
    output = cv2.cvtColor(output, cv2.COLOR_RGB2BGR)  # RGB에서 BGR로 변환
    cv2.imwrite(output_path, output)
    
    print(f"업스케일링된 이미지가 {output_path}에 저장되었습니다.")

if __name__ == "__main__":
    input_path = r'C:\pypjt\images\hugging.jpg'  # 입력 이미지 경로
    output_path = "upscaled_image.png"  # 출력 이미지 경로
    model_path = r"C:\pypjt\restore\Lib\site-packages\Real-ESRGAN\weights\RealESRGAN_x4plus.pth"  # Real-ESRGAN 모델 파일 경로
    
    upscale_image(input_path, output_path, model_path, scale=4)

In [ ]:
import cv2
from basicsr.archs.rrdbnet_arch import RRDBNet
from realesrgan import RealESRGANer
from insightface.utils import face_align

# RealESRGAN을 이용한 업스케일링
model = RRDBNet(num_in_ch=3, num_out_ch=3, num_feat=64, num_block=23, num_grow_ch=32, scale=2)
upsampler = RealESRGANer(
    scale=2, model_path='weights/RealESRGAN_x4plus.pth', model=model, tile=400, tile_pad=10, pre_pad=0, half=True
)

# 이미지 불러오기
image = cv2.imread(r'C:\pypjt\images\hugging.jpg')

# 리사이즈된 이미지를 업스케일링
upscaled_image, _ = upsampler.enhance(image, outscale=2)

# 결과 저장
cv2.imwrite('upscaled_image.jpg', upscaled_image)


In [1]:
import torch
import cv2
import numpy as np
from basicsr.archs.rrdbnet_arch import RRDBNet
from basicsr.utils.realesrgan_utils import RealESRGANer
from typing import Literal

UPSCALE_MODEL_X2 = r"C:\models\RealESRGAN_x2plus.pth"
UPSCALE_MODEL_X4 = r"C:\models\RealESRGAN_x4plus.pth"

def upscale_image(input_image: np.ndarray, scale: Literal[2, 4] = 2) -> np.ndarray:
    """
    이미지를 업스케일링하는 함수.
    
    :param input_image: ndarray 타입의 입력 이미지
    :param model_path: 모델 파일 경로
    :param scale: 업스케일 배율 (기본값: 2)
    :return: 업스케일링된 ndarray 타입의 이미지
    """
    # 장치 설정
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        
    # Upscaing Model 선택
    model_path = UPSCALE_MODEL_X2 if scale == 2 else UPSCALE_MODEL_X4

    # 모델 생성
    model = RRDBNet(num_in_ch=3, num_out_ch=3, num_feat=64, num_block=23, num_grow_ch=32, scale=scale)
    
    # 업스케일러 생성 (half=False로 설정하여 FP32 사용)
    upscaler = RealESRGANer(scale=scale, model_path=model_path, model=model, tile=400, tile_pad=10, pre_pad=0, half=False)
    
    # 입력 이미지가 BGR 형식일 경우, RGB로 변환
    if input_image.shape[-1] == 3:  # 이미지가 컬러일 경우
        input_image = cv2.cvtColor(input_image, cv2.COLOR_BGR2RGB)
    
    # 업스케일링 수행
    output, _ = upscaler.enhance(input_image, outscale=scale)
    
    # 결과를 BGR로 다시 변환하여 반환
    output = cv2.cvtColor(output, cv2.COLOR_RGB2BGR)
    
    return output

In [ ]:
import cv2
import numpy as np

input = r"F:\Windows64\Studio\[Urabon]\asnws-792.+\gin50.jpg"
# 입력 이미지 로드
input_img = cv2.imread(input)

# upscaling 함수 호출
upscaled_img = upscale_image(input_img, scale=2)

# 결과 저장
output = r"F:\Windows64\Studio\[Urabon]\asnws-792.+\up_gin50.jpg"
cv2.imwrite(output, upscaled_img)

In [3]:
import cv2
import torch
from torchvision.transforms.functional import normalize
from basicsr.utils import img2tensor, tensor2img
from facelib.utils.face_restoration_helper import FaceRestoreHelper
from basicsr.utils.registry import ARCH_REGISTRY

# 모델 경로 설정
CODEFORMER_MODEL = r"C:\Users\tanmi\stable-diffusion-webui\models\Codeformer\codeformer-v0.1.0.pth"

def restore_face(input_image, use_gpu=False):
    """
    얼굴 복원 함수
    
    :param input_image: ndarray 타입의 입력 이미지
    :param model_path: CodeFormer 모델 파일 경로
    :param use_gpu: GPU 사용 여부 (기본값: False)
    :return: ndarray 타입의 복원된 이미지
    """
    # 모델 로드
    device = torch.device('cuda' if use_gpu and torch.cuda.is_available() else 'cpu')
    model = ARCH_REGISTRY.get('CodeFormer')(dim_embd=512, codebook_size=1024, n_head=8, n_layers=9, connect_list=['32', '64', '128', '256']).to(device)
    
    checkpoint = torch.load(CODEFORMER_MODEL, weights_only=True, map_location=device)['params_ema']
    model.load_state_dict(checkpoint)
    model.eval()

    # 이미지 크기 저장
    h, w, _ = input_image.shape

    # 얼굴 검출 및 정렬
    face_helper = FaceRestoreHelper(
        upscale_factor=1,
        face_size=512,
        crop_ratio=(1, 1),
        det_model='retinaface_resnet50',
        save_ext='png',
        use_parse=True,
        device=device
    )
    face_helper.read_image(input_image)
    face_helper.get_face_landmarks_5(only_center_face=False, resize=640, eye_dist_threshold=5)
    face_helper.align_warp_face()

    # 얼굴 복원
    for idx, cropped_face in enumerate(face_helper.cropped_faces):
        cropped_face_t = img2tensor(cropped_face / 255., bgr2rgb=True, float32=True)
        normalize(cropped_face_t, (0.5, 0.5, 0.5), (0.5, 0.5, 0.5), inplace=True)
        cropped_face_t = cropped_face_t.unsqueeze(0).to(device)

        try:
            with torch.no_grad():
                output = model(cropped_face_t, w=0.5, adain=True)[0]
                restored_face = tensor2img(output, rgb2bgr=True, min_max=(-1, 1))
            del output
            if use_gpu:
                torch.cuda.empty_cache()
        except RuntimeError as error:
            print(f'Error: {error}')
            print('If you encounter CUDA out of memory, try to set --tile with a smaller number.')
        else:
            restored_face = restored_face.astype('uint8')
            face_helper.add_restored_face(restored_face)

    # 결과 생성
    face_helper.get_inverse_affine(None)
    restored_img = face_helper.paste_faces_to_input_image()
    
    # 최종 이미지 크기 조정 (원본 크기로)
    restored_img = cv2.resize(restored_img, (w, h))

    return restored_img

In [ ]:
import cv2
import numpy as np

input = r"F:\Windows64\Studio\[Urabon]\asnws-792.+\up_gin50.jpg"
# 입력 이미지 로드
input_img = cv2.imread(input)

# upscaling 함수 호출
upscaled_img = restore_face(input_img)

# 결과 저장
output = r"F:\Windows64\Studio\[Urabon]\asnws-792.+\re_up_gin50.jpg"
cv2.imwrite(output, upscaled_img)

In [ ]:
import cv2
import numpy as np

input = r"F:\Windows64\Studio\[Urabon]\asnws-792.+\gin49.jpg"
# 입력 이미지 로드
input_img = cv2.imread(input)

# upscaling 함수 호출
upscaled_img = upscale_image(input_img, scale=2)
# restoring 함수 호출
restored_img = restore_face(upscaled_img)

# 결과 저장
output = r"F:\Windows64\Studio\[Urabon]\asnws-792.+\re_up_gin49.jpg"
cv2.imwrite(output, restored_img)

In [ ]:
import cv2
import os
import numpy as np

# 입력 디렉토리와 출력 디렉토리
input_dir = r"F:\[Fake]\[facing]\multi"
output_dir = r"F:\[Fake]\[facing]\multi\restored"

# 출력 디렉토리가 없으면 생성
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# 디렉토리 내의 모든 파일 처리
for filename in os.listdir(input_dir):
    if filename.endswith((".jpg", ".jpeg", ".png")):  # 이미지 파일 확장자 체크
        input_path = os.path.join(input_dir, filename)

        # 입력 이미지 로드
        input_img = cv2.imread(input_path)

        if input_img is not None:  # 이미지가 정상적으로 로드되었는지 확인
            # upscaling 함수 호출
            upscaled_img = upscale_image(input_img, scale=2)

            # restoring 함수 호출
            restored_img = restore_face(upscaled_img)

            # 결과 저장
            output_path = os.path.join(output_dir, f"re_up_{filename}")
            cv2.imwrite(output_path, restored_img)
            print(f"Processed and saved: {output_path}")
        else:
            print(f"Failed to load: {input_path}")

In [ ]:
import cv2
import os
import numpy as np

# 입력 디렉토리와 출력 디렉토리
input_dir = r"F:\[Fake]\[facing]\multi"
output_dir = r"F:\[Fake]\[facing]\multi\restored"

# 출력 디렉토리가 없으면 생성
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# 디렉토리 내의 모든 파일 처리
for filename in os.listdir(input_dir):
    if filename.endswith((".jpg", ".jpeg", ".png")):  # 이미지 파일 확장자 체크
        input_path = os.path.join(input_dir, filename)

        # 입력 이미지 로드
        input_img = cv2.imread(input_path)

        if input_img is not None:  # 이미지가 정상적으로 로드되었는지 확인
            
            # restoring 함수 호출
            restored_img = restore_face(input_img)

            # 결과 저장
            output_path = os.path.join(output_dir, f"re_up_{filename}")
            cv2.imwrite(output_path, restored_img)
            print(f"Processed and saved: {output_path}")
        else:
            print(f"Failed to load: {input_path}")